In [2]:
from typing import List, TypedDict
from langchain import hub
from langchain_core.documents import Document
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma
# from langgraph.graph import START, StateGraph
from sentence_transformers import SentenceTransformer
import torch

# 0) Qwen-3-8B LLM 세팅
# HuggingFace 파이프라인을 이용해 Qwen-3-8B 모델 로드
# model_name = "Qwen/Qwen3-8B"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.float16,
#     device_map="auto"
# )
# text_gen = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     trust_remote_code=True,
#     max_new_tokens=1024
# )
# # LangChain HuggingFacePipeline 래퍼 생성
# llm = HuggingFacePipeline(pipeline=text_gen)

# # 1) VectorStore 세팅
# class MyEmbeddings:
#     def __init__(self, model):
#         self.model = model

#     def embed_documents(self, texts: List[str]) -> List[List[float]]:
#         return self.model.encode(texts, convert_to_numpy=True).tolist()

#     def embed_query(self, text: str) -> List[float]:
#         # for a single query you can still use encode
#         return self.model.encode([text], convert_to_numpy=True)[0].tolist()

# from sentence_transformers import SentenceTransformer
# embed_model = SentenceTransformer("dragonkue/snowflake-arctic-embed-l-v2.0-ko")
# embeddings = MyEmbeddings(embed_model)
# vectordb = Chroma(persist_directory="../chroma_db", embedding_function=embeddings,collection_name="construction")

In [3]:
from typing import List, TypedDict
from langchain import hub
from langchain_core.documents import Document
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma
# from langgraph.graph import START, StateGraph
from sentence_transformers import SentenceTransformer
import torch

class MyEmbeddings:
    def __init__(self, model):
        self.model = model

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self.model.encode(texts, convert_to_numpy=True).tolist()

    def embed_query(self, text: str) -> List[float]:
        # for a single query you can still use encode
        return self.model.encode([text], convert_to_numpy=True)[0].tolist()

from sentence_transformers import SentenceTransformer
embed_model = SentenceTransformer("dragonkue/snowflake-arctic-embed-l-v2.0-ko")
embeddings = MyEmbeddings(embed_model)
vectordb = Chroma(persist_directory="./chroma_db", embedding_function=embeddings,collection_name="construction")

test_query = "램프 경사"
    # k=3개 문서 검색
retrieved_docs = vectordb.similarity_search(test_query, k=3)

print("=== 검색 결과 ===")
if not retrieved_docs:
    print("문서가 없습니다. 데이터 경로와 임베딩된 문서를 확인하세요.")
else:
    for idx, doc in enumerate(retrieved_docs, start=1):
        src = doc.metadata.get("source", "unknown")
        snippet = doc.page_content.replace("\n", " ")[:200]
        print(f"{idx}. [{src}] {snippet}...")

=== 검색 결과 ===
1. [unknown] 램프 경사 레벨 확인 +36레벨에서 +40레벨로 비상차량 이동시 +39에서 +40 경사가 급해보이므로 경사 레벨 확인 필요(또는 균등 배분) 램프 길이 1,000 → 1,300 경사구배 1/10 → 1/13 완화하였음 비상차량동선 경사 구배에 문제 없도록 수정됨 검증의견에 대한 답변 작성시 관련 근거(LH관련지침, 관련법, 민간사례, 관련 지자체 협의여부 ...
2. [unknown] 램프바닥 방수마감 표기 누락 램프바닥 방수마감 표기 누락 램프 바닥 방수 마감 표기    ...
3. [unknown] 지하주차장 램프 진입구 외부방향으로 역구배 미반영 (체크리스트 A35) 램프 진입구 외부구배 1/10 미반영  역구배 구간 추가 표기 역구배 표시 또는 기울기 표기 바람   역구배 표시 또는 기울기 표기 바람...
